# 函数参数
## 1 位置参数(positional argument)
- 固定数量位置参数
- 可变数量位置参数(varargs/star args/*args)
## 2 关键字参数(keyword argument)
优点：可读性更强、可以带默认值
注意：关键字参数总是应该通过参数名而不是位置来传递（如果不这么做会使可读性变差）
- 固定数量关键字参数
- 可变数量关键字参数(/**kwargs)
## 3 默认值会变的参数
使用None或docstring来描述默认值会变的参数
## 4 清晰的参数列表
使用只能以关键字指定和只能按位置传入的参数来设计清晰的参数列表

In [3]:
# 固定数量位置参数
def log(message, values):
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', [1, 2])
log('Hi there', []) # 必须要传递一个参数，否则会报参数数量不匹配的错误

My numbers are: 1, 2
Hi there


In [7]:
# *args 可变数量位置参数
def log(message, *values):  # The only difference
    if not values:
        print(message)
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{message}: {values_str}')

log('My numbers are', 1, 2)
log('Hi there')  # Much better

# 将已有序列当成参数传给函数，可以使用*操作符
favorites = [7, 33, 99]
log('Favorite colors', *favorites)

# 但是会导致两个问题
# - 1 这些参数会先转化为元组，也就意味着传递的是迭代器，必须迭代所有元素才能向下执行，会耗用大量内存
def my_generator():
    for i in range(10):
        yield i

def my_func(*args):
    print(args)

it = my_generator()
my_func(*it)

# - 2 使用*args后，又新增了位置参数，原有调用需要全部更新，不然都会报错
def log(sequence, message, *values):
    if not values:
        print(f'{sequence} - {message}')
    else:
        values_str = ', '.join(str(x) for x in values)
        print(f'{sequence} - {message}: {values_str}')

log(1, 'Favorites', 7, 33)      # New with *args OK
log(1, 'Hi there')              # New message only OK
log('Favorite numbers', 7, 33)  # 旧调用出错

My numbers are: 1, 2
Hi there
Favorite colors: 7, 33, 99
(0, 1, 2, 3, 4, 5, 6, 7, 8, 9)
1 - Favorites: 7, 33
1 - Hi there
Favorite numbers - 7: 33


In [9]:
# 固定数量关键字参数
def remainder(number, divisor):
    return number % divisor

print(remainder(20, 7)) # 按照位置传递
print(remainder(20, divisor=7)) # 混用时，位置参数在关键字参数前
print(remainder(number=20, divisor=7)) # 关键字参数的顺序无所谓
print(remainder(divisor=7, number=20))

6
6
6
6


In [12]:
# **kwargs 可变数量关键字参数

# **运算符，可以把字典里面的键值以关键字参数的形式传递给函数 
my_kwargs = {
	'number': 20,
	'divisor': 7,
}
assert remainder(**my_kwargs) == 6

# 可以和位置参数混用，但是不要重复指定
my_kwargs = {
	'divisor': 7,
}
assert remainder(number=20, **my_kwargs) == 6

# 多个字典也可以，同样只要不重复指定就行
my_kwargs = {
	'number': 20,
}
other_kwargs = {
	'divisor': 7,
}
assert remainder(**my_kwargs, **other_kwargs) == 6

# 定义函数时，可以用万能形参**kwargs来接收参数到一个字典里备用
def print_parameters(**kwargs):
    for key, value in kwargs.items():
        print(f'{key} = {value}')

print_parameters(alpha=1.5, beta=9, gamma=4)

alpha = 1.5
beta = 9
gamma = 4


In [20]:
# 默认值会变的参数
from time import sleep
from datetime import datetime

def log(message, when=datetime.now()):
    print(f'{when}: {message}')

log('Hi there!')
sleep(0.1)
log('Hello again!')
# 形参只会计算一次
# 2022-11-18 05:40:03.625367: Hi there!
# 2022-11-18 05:40:03.625367: Hello again!

# 默认值会变的参数，使用None赋值，并在docstring文档中说明接下来会怎么变化
def log(message, when=None):
    """Log a message with a timestamp.
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.
    """
    if when is None:
        when = datetime.now()
    print(f'{when}: {message}')

log('Hi there!')
sleep(0.1)
log('Hello again!')

help(log) # 读取log的docstring

2022-11-18 05:44:42.832715: Hi there!
2022-11-18 05:44:42.832715: Hello again!
2022-11-18 05:44:42.945663: Hi there!
2022-11-18 05:44:43.053837: Hello again!
Help on function log in module __main__:

log(message, when=None)
    Log a message with a timestamp.
    Args:
        message: Message to print.
        when: datetime of when the message occurred.
            Defaults to the present time.



In [23]:
# 表示以后会被调用者修改的参数，初始化使用None
import json

def decode(data, default=None):
    """Load JSON data from a string.
    Args:
        data: JSON data to decode.
        default: Value to return if decoding fails.
            Defaults to an empty dictionary.
    """
    try:
        return json.loads(data)
    except ValueError:
        if default is None:
            default = {}
        return default

foo = decode('bad data')
foo['stuff'] = 5
bar = decode('also bad')
bar['meep'] = 1
print('Foo:', foo)
print('Bar:', bar)
assert foo is not bar

Foo: {'stuff': 5}
Bar: {'meep': 1}


In [30]:
# ‘/’前只能按位置传入（python3.8以后的新特性），‘*’后只能按关键字传入，中间的两种方式都可以
def safe_division_e(numerator, denominator, /,
                    ndigits=10, *,                # Changed
                    ignore_overflow=False,
                    ignore_zero_division=False):
    try:
        fraction = numerator / denominator        # Changed
        return round(fraction, ndigits)           # Changed
    except OverflowError:
        if ignore_overflow:
            return 0
        else:
            raise
    except ZeroDivisionError:
        if ignore_zero_division:
            return float('inf')
        else:
            raise

result = safe_division_e(22, 7)
print(result)

result = safe_division_e(22, 7, 5)
print(result)

result = safe_division_e(22, 7, ndigits=2)
print(result)

result = safe_division_e(22, 0, 2, ignore_zero_division=True) 
print(result)

result = safe_division_e(22, 0, 2, False, ignore_zero_division=True) 
# 报错，应该是2到3个位置参数，你传了4个
# takes from 2 to 3 positional arguments but 4 positional arguments

3.1428571429
3.14286
3.14
inf


TypeError: safe_division_e() takes from 2 to 3 positional arguments but 4 positional arguments (and 1 keyword-only argument) were given